In [2]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

In [18]:
# input dataset
df = pd.read_csv('Dataset_Manual_stemming18k_baru.csv', encoding='latin1', delimiter=';')
df.head()

,Pilpres 2024,Type,Mentions,Date,Media,Sentiment,Author,Followers,Retweeted,Favourited,Mentions1,stemming
0,36600,rt,RT @bachrum_achmadi: Sblm Ganjar resmi capres ...,27/05/2023 13:16,Twitter,Positive,@krisnugrohoadi (iBoBig),94.0,39.0,0.0,sblm ganjar resmi capres pdip bawaslu bkn main...,sblm ganjar resmi capres pdip bawaslu bkn main...
1,63886,rt,RT @P3n99u94t: ANIES BASWEDAN CAPRES TERLENGKA...,23/05/2023 16:47,Twitter,Neutral,@DezzarZo4 (DeZZarZo4 ÃÂÃÂÃÂÃÂ°ÃÂÃÂ...,1057.0,24.0,0.0,anies baswedan capres terlengkap,anies baswedan capres lengkap
2,40994,rt,RT @Anak__Ogi: Buat @Pak_JK pendukung capres N...,27/05/2023 05:32,Twitter,Positive,@Toms32B (Toms 32b),42.0,218.0,0.0,buat pendukung capres nasdem demokrat dan pks,buat dukung capres nasdem demokrat dan pks
3,63245,rt,RT @Chaves1305: Capres @prabowo emang sangat k...,23/05/2023 17:35,Twitter,Positive,@Chaves1305 (Chaves),340.0,1.0,0.0,capres emang sangat konsisten utk indonesia,capres emang sangat konsisten utk indonesia
4,64087,mention,Para Kiai Besar Dukung Prabowo-Gus Muhaimin di...,23/05/2023 14:31,Twitter,Positive,@gadotaja (hanyamanusia),110.0,0.0,0.0,para kiai besar dukung prabowogus muhaimin di ...,para kiai besar dukung prabowogus muhaimin di ...


In [20]:
# input kolom yang akan dipakai
texts = df['stemming'].astype(str).tolist()

In [12]:
# Load model
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [13]:
# Fungsi prediksi sentimen
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True, max_length=128)
    outputs = model(inputs)
    logits = outputs.logits
    predictions = tf.nn.softmax(logits, axis=-1)
    label = tf.argmax(predictions, axis=1).numpy()[0]
    
    # Gabungkan label menjadi positif (1) dan negatif (0)
    if label in [0, 1]:
        return 'negatif'  # Negatif
    elif label in [2, 3, 4]:
        return 'positif'  # Positif

In [14]:
# Prediksi sentimen
predicted_labels = [predict_sentiment(text) for text in texts]

In [15]:
# menambahkan hasil prediksi ke dataset
df['label'] = predicted_labels

In [16]:
# simpan dataset baru
labeled_dataset_path = 'labelterbaruu_dataset.csv'
df.to_csv(labeled_dataset_path, index=False)